In [1]:
%pylab inline

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


## Libraries

In [2]:
import numpy as np
from numpy.random import randn

import pandas as pd

#time
from datetime import datetime
from datetime import timedelta

#counting
from collections import Counter

In [3]:
# good old matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

#high-level based on matplotlib
import seaborn as sns

#dynamic and interactive
import highcharts
from highcharts.charts import chart

## Init

In [4]:
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
np.random.seed(1234)

In [5]:
#loading highcharts javascript
highcharts.init()

## Reading data

### Checkins

In [23]:
dfc = pd.read_csv('../../datasets/loc-gowalla_totalCheckins.txt', sep='\t', header=False)
dfc.columns = ['uid','utc','lat','lon','vid']

dfc['utc'] = dfc['utc'].astype('datetime64[ms]')

dfc.head()

,uid,utc,lat,lon,vid
0,0,2010-10-18 22:17:43,30.269103,-97.749395,420315
1,0,2010-10-17 23:42:03,30.255731,-97.763386,316637
2,0,2010-10-17 19:26:05,30.263418,-97.757597,16516
3,0,2010-10-16 18:50:42,30.274292,-97.740523,5535878
4,0,2010-10-12 23:58:03,30.261599,-97.758581,15372


### Data munging: cleanup, time parsing, etc.

New York, New York!
 - bounding box: (40.4774, -74.2589), (40.9176, -73.7004)

In [24]:
ny = (dfc['lat']>=40.4774) & (dfc['lat']<=40.9176) & (dfc['lon']>=-74.2589) & (dfc['lon']<=-73.7004)
dfc = dfc[ny]

Time columns

In [25]:
dfc['year']     = dfc['utc'].apply(lambda x: x.date().year)
dfc['month']    = dfc['utc'].apply(lambda x: x.date().month)
dfc['day']      = dfc['utc'].apply(lambda x: x.date().day)

dfc['date']     = dfc['utc'].apply(lambda x: x.date())
dfc['time']     = dfc['utc'].apply(lambda x: x.time())

dfc['isotime']  = dfc['utc'].apply(lambda x: '0000-00-00T' + x.time().isoformat() +'Z')

dfc = dfc[dfc['date']<=datetime(2010,10,19).date()]
dfc[['uid','utc','lat','lon','vid', 'time', 'isotime']].head()

,uid,utc,lat,lon,vid,time,isotime
9,0,2010-10-12 00:21:28,40.643885,-73.782806,23261,00:21:28,0000-00-00T00:21:28Z
10,0,2010-10-11 20:21:20,40.741374,-73.988105,16907,20:21:20,0000-00-00T20:21:20Z
11,0,2010-10-11 20:20:42,40.741388,-73.989455,12973,20:20:42,0000-00-00T20:20:42Z
12,0,2010-10-11 00:06:30,40.724910,-73.994621,341255,00:06:30,0000-00-00T00:06:30Z
13,0,2010-10-10 22:00:37,40.729768,-73.998535,260957,22:00:37,0000-00-00T22:00:37Z


### Venues

In [26]:
dfv = pd.read_csv('../../datasets/spots.txt', sep='\t', header=False)
dfv.columns = ['vid','name','loc']

coords = dfv['loc'].replace('[^0-9. -]+', '',regex=True)
coords = coords.apply(lambda x: x.split())

dfv['v_lat'] = coords.apply(lambda x: float(x[1]))
dfv['v_lon'] = coords.apply(lambda x: float(x[0]))

dfv = dfv.drop('loc', 1)

ny = (dfv['v_lat']>=40.4774) & (dfv['v_lat']<=40.9176) & (dfv['v_lon']>=-74.2589) & (dfv['v_lon']<=-73.7004)
dfv = dfv[ny]

dfv.head()

,vid,name,v_lat,v_lon
0,1391604,Conference House Park,40.501759,-74.252343
1,1391611,Almer G. Russell Pavilion,40.502265,-74.254264
2,3612422,Conference House,40.500064,-74.249042
3,3612431,Billop House,40.500064,-74.249042
4,1391499,Biddle House,40.505483,-74.254146


### Merge venue names and checkins

In [27]:
df = pd.merge(dfc, dfv[['vid', 'name']], how='left', on='vid')

Missing venue names on the available checkins:

In [28]:
print "missing venue names: {}% of available checkins".format(len(df[pd.isnull(df['name'])])*100 / len(df))

missing venue names: 19% of available checkins


## Data Exploration

In [29]:
d = df.groupby('date').size()

In [30]:
chart({
            'chart': {
                'type': 'line',
                'marginRight': 30,
                'marginBottom': 50
            },
            'title': {
                'text':'#checkins per day in New York City'
            },
            'yAxis': {
                'type': 'linear',
                'title': {'text':'#checkins'},
                
            },
            'xAxis': {
                'categories': [str(x) for x in d.index.tolist()]
            },
            'series': [{
                'name': 'date',
                'data': d.tolist()
            }]
})

## Clustering

In [31]:
from sklearn.cluster import KMeans

In [32]:
start = datetime(2010,1,1,0,0) 
dfw = df[df['utc']>start]

In [34]:
cl = min(200, len(dfw)/8)
ml = KMeans(n_clusters=cl)
ml.fit(dfw[['lat', 'lon']])

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=200, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [35]:
labels = Counter(ml.labels_)
clusters = [ [ ml.cluster_centers_[x][1], ml.cluster_centers_[x][0], labels[x] ] for x in labels]
clusters

[[-73.978606279031723, 40.763183528234528, 2317],
 [-73.809981313318033, 40.70100718954118, 255],
 [-74.173764814744004, 40.669625361488002, 100],
 [-74.014700563498479, 40.702718732265147, 789],
 [-74.099442531152022, 40.830082782405405, 148],
 [-73.872682901571764, 40.774579438142617, 2100],
 [-73.988497998382087, 40.743826994971229, 2892],
 [-73.946926002322982, 40.828310549134784, 322],
 [-73.981521604067154, 40.674563518350318, 1105],
 [-73.78159774862776, 40.644712698468545, 2489],
 [-74.177273970713685, 40.744649270093454, 336],
 [-73.961440548727694, 40.715815647616118, 1365],
 [-73.880636767574089, 40.858108095920649, 247],
 [-73.979827164176186, 40.574333011632696, 315],
 [-74.193059973229623, 40.871332523172221, 54],
 [-73.996382097033603, 40.723354283283335, 3018],
 [-74.109581690630876, 40.629892160678246, 285],
 [-74.029598273576681, 40.907079519863679, 223],
 [-73.954548564480646, 40.784450302896076, 1173],
 [-74.164743244323105, 40.57854353444089, 225],
 [-73.7839002855

In [37]:
delta=0.15
chart({
            'chart': {
                'type': 'bubble',
                'zoomType': 'xy',
                'width':800,
                'height':800,
                'plotBackgroundImage':'https://maps.googleapis.com/maps/api/staticmap?center=40.8,-74.0&zoom=11&size=800x800&maptype=roadmap'
            },
            'plotOptions': {
              'bubble': {
                'maxSize':'5%',
                'minSize':'1%'
               }
             },
            'legend': { 
              'enabled': False
            },
            'yAxis': {
              'min':40.8-delta,
              'max':40.8+delta
            },
            'xAxis': {
              'min':-74.0-delta,
              'max':-74.0+delta
            },
            'series': [{'data': clusters, 'color':"#FF0000"}]
}, '800px', '800px')

### Write data to csv file

####checkins

In [38]:
cols = ['year','month','day','isotime','uid','lat','lon','vid']
dfc[cols].to_csv('../../datasets/checkins.csv', header=False, index=False)

####venues

In [39]:
dfv[['vid','name','v_lat','v_lon']].to_csv('../../datasets/venues.csv', header=False, index=False)